# Лабораторная работа по анализу Twitter

Целью лабораторной работы является научиться применять методы обучения с учителем (supervised learning) для задачи анализа тональностей, а также освоить специализированный инструментарий для компьютерной лингвистики и машинного обучения.

- Загрузить с использованием csv файлов твиты.
- Провести предобработку корпуса (удаление стоп-слов, и т.д.)
- Используя выбранную модель репрезентации получить матричное представление.
- Используя изученные классификаторы получить численные оценки точности, полноты и F-меры.
- Объяснить полученные результаты.

In [49]:
import os
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.EMOJI,p.OPT.SMILEY,p.OPT.NUMBER,p.OPT.HASHTAG)

import warnings
warnings.simplefilter('ignore')

## Загрузка и первичный анализ данных

* Выгрузим все данные, объединим их в один датафрейм и посмотрим на них.

In [50]:
positive = pd.read_csv("positive.csv", ";", header=None)
negative = pd.read_csv("negative.csv", ";", header=None)
result = pd.concat([positive, negative])

In [51]:
result.columns = ['id', 'tdate', 'tname', 'ttext', 'polarity', 'trep', 'trtw', 'tfav', 'tstcount', 'tfol', 'tfriends', 'listcount']

In [52]:
result = result.sample(frac=1).reset_index(drop=True)
result.head()

,id,tdate,tname,ttext,polarity,trep,trtw,tfav,tstcount,tfol,tfriends,listcount
0,415523764130959360,1387903560,Pautova96,"Сезон ""мандоринопоедание"",""а где подарки"",""сал...",-1,0,0,0,265,24,52,0
1,410395424386609152,1386680868,KabanMadan,@h197009 @VaDima32 вот поэтому я никогда не иг...,1,0,0,0,35011,699,831,11
2,411388687251820544,1386917680,katawo,"@Kapna к сожалению, хуевый концерт. Я была раз...",-1,0,0,0,8906,272,107,10
3,410485347999842304,1386702308,yanhik_,@HaliullinaDiana мы можем просто попеть:)\nГлу...,1,0,0,0,4586,73,65,3
4,419151425893986305,1388768462,No_no_nobody,@RussianIdiot99 Я тоже реву(( бля вот грустный...,-1,0,0,0,19386,477,310,10


Теперь разобьем наш набор данных случайным образом на train и test выборки с отношением 80/20

In [53]:
train_data, test_data = train_test_split(result, test_size=0.2, random_state=42)

Для дальнейшего анализа нам нужны только текст каждого твита и его категория, поэтому удаляем все остальные колонки

In [54]:
train_data = train_data.drop(['id', 'tdate', 'tname', 'trep', 'trtw', 'tfav', 'tstcount', 'tfol', 'tfriends', 'listcount'], axis = 1)
test_data = test_data.drop(['id', 'tdate', 'tname', 'trep', 'trtw', 'tfav', 'tstcount', 'tfol', 'tfriends', 'listcount'], axis = 1)

In [55]:
train_data.head()

,ttext,polarity
107097,"Хочу в нашем городе увидеться с любимой, но у ...",-1
224163,@30GREENROMANCE ты милый :3 м не спорь со стар...,1
63992,Сука снега овер дохуя гараж боя чистить надо.....,-1
108549,@Nika_RulesWorld та он как-то медленно работает((,-1
67962,"@Toks3000 @treningi2 \nСергей , по поводу курс...",-1


In [56]:
test_data.head()

,ttext,polarity
215081,RT @natalya_stails: @ccrimsoncupcake бедняшка(...,-1
220754,"Как новый год,так у меня чистый лист в голове ...",-1
60463,RT @Shinebaatar: Өө хөөрхий энэ хаягийг ингэж ...,-1
213263,Дай обезьяне телефон.) Прекрасная ночь с неожи...,1
113055,RT @KorelinG: Наши биатлонисточки просто сказо...,1


- Теперь, когда данные приведены к единому виду, необходимо выполнить предобработку текста: привести текст к нижнему регистру, очистить его от всего лишнего - спец символов, чисел, ссылок и зарезервированных слов.

## Предобработка текста

* Возьмем твит из тренировочной выборки и посмотрим на него.

In [57]:
train_data['ttext'][7]

'еже лох — знаю((((((((( http://t.co/EzAEqYXQVK'

* Текст необходимо очистить. Для этого воспользуемся библиотекой для предобработки твитов - https://github.com/s/preprocessor  
Ее особенность в том, что она позволяет удалять ссылки, хэштеги, упоминания и т.д. 
Код применения ниже:

In [58]:
p.clean(train_data['ttext'][7].lower())

'еже лох — знаю((((((((('

* Теперь воспользуемся токенайзером, который очистит текст от оставшихся ненужных символов и соединим получившиеся токены в предложения

In [59]:
' '.join(CountVectorizer().build_tokenizer()(p.clean(train_data['ttext'][7].lower())))

'еже лох знаю'

* Теперь сделаем то же самое со всеми данными

In [60]:
train_data['ttext'] = train_data['ttext'].apply(lambda t: ' '.join(CountVectorizer().build_tokenizer()(p.clean(t.lower()))))
test_data['ttext'] = test_data['ttext'].apply(lambda t: ' '.join(CountVectorizer().build_tokenizer()(p.clean(t.lower()))))

Теперь удалим стоп-слова из наших данных

In [61]:
stop_words = set(stopwords.words("russian"))  # load stopwords
train_data['ttext'] = train_data['ttext'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
test_data['ttext'] = test_data['ttext'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [62]:
train_data.head()

,ttext,polarity
107097,хочу нашем городе увидеться любимой выйдет,-1
224163,милый спорь старшими,1
63992,сука снега овер дохуя гараж боя чистить всё пр...,-1
108549,та медленно работает,-1
67962,сергей поводу курса фоточтение готов купить то...,-1


In [63]:
test_data.head()

,ttext,polarity
215081,rt бедняшка врачу сходить,-1
220754,новый год чистый лист голове плане подарков зн...,-1
60463,rt өө хөөрхий энэ хаягийг ингэж удирдах гэж дээ,-1
213263,дай обезьяне телефон прекрасная ночь неожиданн...,1
113055,rt наши биатлонисточки просто сказочно летят л...,1


## TF-IDF представление твитов

* Далее, представим твиты в виде TF-IDF. Для этого, воспользуемся TfidfVectorizer'ом.
* Установим значения min_df = 5, чтобы отбросить слова, которые появляются реже 5 раз.

In [64]:
TF_IDF = TfidfVectorizer()
TF_IDF.fit(list(train_data['ttext']) + list(test_data['ttext']))
X_train = TF_IDF.transform(list(train_data['ttext']))
X_test = TF_IDF.transform(list(test_data['ttext']))

y_train = list(train_data['polarity'])
y_test = list(test_data['polarity'])

### *Логистическая регрессия *

In [71]:
model = LogisticRegression()
model.fit(X_train, y_train)

prediction = model.predict(X_test)

In [72]:
print(metrics.classification_report(prediction,y_test,target_names=['-1', '1']))

             precision    recall  f1-score   support

         -1       0.69      0.75      0.72     20691
          1       0.78      0.72      0.75     24676

avg / total       0.74      0.74      0.74     45367



### *Наивный Байесовский классификатор*

In [73]:
model = BernoulliNB()
model.fit(X_train, y_train)

prediction = model.predict(X_test)

In [74]:
print(metrics.classification_report(prediction,y_test,target_names=['-1', '1']))

             precision    recall  f1-score   support

         -1       0.77      0.71      0.74     24308
          1       0.69      0.75      0.72     21059

avg / total       0.73      0.73      0.73     45367



### *Метод опорных векторов*

In [75]:
model = LinearSVC()
model.fit(X_train, y_train)

prediction = model.predict(X_test)

* *linear*

In [76]:
print(metrics.classification_report(prediction,y_test,target_names=['-1', '1']))

             precision    recall  f1-score   support

         -1       0.72      0.73      0.72     22103
          1       0.74      0.73      0.73     23264

avg / total       0.73      0.73      0.73     45367



## Выводы по алгоритмам

Как можно видеть из результатов, качество всех алгоритмов находится на примерно одинаковом уровне.

Logistic Regression показывает себя чуть лучше остальных, но не существенно. Изменения параметра `C` не привели к каким-либо значимым изменениям.

В виду того, что размерность данных слишком большая для анализа, другие алгоритмы классификации не просчитались до конца на моем компьютере.